In [ ]:
import pandas as pd
import random
import time
import re
from transformers import pipeline
import difflib

chatbot = pipeline("text-generation", model="gpt2")

gift_df = pd.read_excel('/content/Cadeaux_Produits (version 1).fr.en.xlsx')

gift_keywords = [
    'gift', 'present', 'surprise'
]

faq_patterns = [
    (r".*how.*recommend.*gift.*", "🎁 I recommend gifts by asking you questions about the recipient, their interests, the occasion, and your budget. Then I match this info with my awesome gift database! ✨"),
    (r".*save.*preference.*", "💾 I'm working on that! Right now, I can't save preferences, but that's a feature coming soon! Stay tuned! 😉"),
    (r".*how.*sure.*gift.*like.*", "🤔 I can't be 100% sure, but I match gifts to interests, age, and occasion to give you the best options! 🎯"),
    (r".*more.*idea.*gift.*", "✨ Of course! Just let me know and I'll show you even more amazing gift ideas! 🎉"),
    (r".*refine.*result.*", "🔍 You can refine your search by giving me more details about the person, their hobbies, or adjusting your budget! 💡")
]

fun_emojis = ['🎁', '✨', '🥳', '🎉', '🎈', '💝', '🛍️', '🎊']
gif_links = [
    "https://i.giphy.com/media/v1.Y2lkPTc5MGI3NjExMHJ1NXdxOXF0MnE0bGw4eXlzZDQwbW14OXd6eXg1NGV6eGUwNHRvOSZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/xTiTnh2PC1qEjDQUrC/giphy.gif",
    "https://i.giphy.com/media/v1.Y2lkPTc5MGI3NjExZjJtczl4bTJwdzczdmJvbGdkNzNmdDlsY2c5NHR2d2o5aXFzbmx0eSZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/7nUkHEdz3AMkkbGWV1/giphy.gif",
]

last_results = pd.DataFrame()

def correct_input(user_input, valid_options):
    closest_match = difflib.get_close_matches(user_input.lower(), valid_options, n=1, cutoff=0.8)
    if closest_match:
        return closest_match[0]
    return user_input

def is_gift_related(message):
    gift_context_keywords = ["buy", "find", "need", "recommend", "idea", "choose"]
    gift_non_context_keywords = ["moment", "live", "natural", "talent", "child", "present day"]

    message = message.lower()

    if any(keyword in message for keyword in gift_keywords):
        if any(non_keyword in message for non_keyword in gift_non_context_keywords):
            return False
        if any(context_keyword in message for context_keyword in gift_context_keywords):
            return True
    return False

def is_faq_related(message):
    for pattern, response in faq_patterns:
        if re.search(pattern, message.lower()):
            return response
    return None

def show_gif():
    print(f"Check this out! {random.choice(gif_links)}")

def ask_gift_questions():
    print(f"{random.choice(fun_emojis)} Let's find the perfect gift for you! {random.choice(fun_emojis)}")

    valid_recipients = ['adult', 'child', 'teenager', 'all age range']
    valid_genders = ['man', 'women', 'unisex']
    valid_occasions = ['holidays', 'birthday', 'partie', 'retirement', "new year's day", 'housewarming', 'wedding', "valentine's day", 'christmas', 'all occasion']
    valid_relationships = ['family', 'friends', 'colleagues', 'Husband/Fiancé/Boyfriend', 'all relationship']
    valid_interest= ['relaxation', 'friendliness', 'adventure', 'art', 'beauty', 'camping', 'cinema', 'collection', 'comfort', 'cooking', 'creativity', 'cuisine', 'culture', 'DIY', 'decoration', 'ecology', 'fashion', 'games', 'gaming', 'gardening', 'gastronomy', 'geography', 'health', 'history', 'home', 'journey', 'kitchen', 'mobility', 'music', 'nature', 'nostalgia', 'office', 'party', 'photography', 'reading', 'security', 'sport', 'technology', 'travel', 'video games', 'well-being', 'all center of interest']

    recipient = correct_input(input("👤 Who is the gift for? (adult, child, teenager, all age range): "), valid_recipients)
    while recipient not in valid_recipients:
        print(f"⚠️ Invalid option. Please choose from {', '.join(valid_recipients)}.")
        recipient = correct_input(input("👤 Who is the gift for? (adult, child, teenager, all age range): "), valid_recipients)

    gender = correct_input(input("🚻 What is the gender of the person? (man, women, unisex): "), valid_genders)
    while gender not in valid_genders:
        print(f"⚠️ Invalid option. Please choose from {', '.join(valid_genders)}.")
        gender = correct_input(input("🚻 What is the gender of the person? (man, women, unisex): "), valid_genders)

    occasion = correct_input(input("🎉 What is the occasion? (holidays, birthday, christmas, retirement, new year's day, housewarming, wedding, valentine's day, all occasion): "), valid_occasions)
    while occasion not in valid_occasions:
        print(f"⚠️ Invalid option. Please choose from {', '.join(valid_occasions)}.")
        occasion = correct_input(input("🎉 What is the occasion? (holidays, birthday, christmas, retirement, new year's day, housewarming, wedding, valentine's day, all occasion): "), valid_occasions)

    relationship = correct_input(input("🤝 What is your relationship with the person? (family, friends, colleagues, Husband/Fiancé/Boyfriend, all relationship): "), valid_relationships)
    while relationship not in valid_relationships:
        print(f"⚠️ Invalid option. Please choose from {', '.join(valid_relationships)}.")
        relationship = correct_input(input("🤝 What is your relationship with the person? (family, friends, colleagues, Husband/Fiancé/Boyfriend, all relationship): "), valid_relationships)

    print(f"Know we will affine the interest of the lucky receiver. Here is the list of the accepted interest:")
    print(f"{', '.join(valid_interest)}")
    print(f"Else you can still 'all center of interest'")
    interest = correct_input(input("💡 What are their interests?: ").lower(), valid_interest)
    while interest not in valid_interest:
        print(f"⚠️ Invalid option. Please choose from {', '.join(valid_interest)}.")
        relationship = correct_input(input("💡 What are their interests?:"), valid_relationships)
    budget = input("💸 What is your maximum budget? (in euros): ")
    return recipient, gender, occasion, relationship, interest, budget

def filter_gifts(recipient, gender, occasion, relationship, interest, budget):
    global last_results
    try:
        budget = float(budget)
    except ValueError:
        print("⚠️ Invalid budget. Using the maximum budget by default.")
        budget = gift_df['Price (€)'].max()

    if gender == 'man':
        gender_filter = ['man', 'unisex']
    elif gender == 'women':
        gender_filter = ['women', 'unisex']
    else:
        gender_filter = ['unisex']

    interest_filter = True if interest == 'all center of interest' else gift_df['Centers of Interest'].str.lower().str.contains(interest.lower(), na=False)
    occasion_filter = True if occasion == 'all occasion' else gift_df['Event'].str.lower().str.contains(occasion.lower(), na=False)
    relationship_filter = True if relationship == 'all relationship' else gift_df['Scope'].str.lower().str.contains(relationship.lower(), na=False)
    recipient_filter = True if recipient == 'all age range' else (gift_df['Age Range'].str.lower() == recipient.lower())

    results = gift_df[
        recipient_filter &
        (gift_df['Sex'].str.lower().isin(gender_filter)) &
        occasion_filter &
        relationship_filter &
        interest_filter &
        (gift_df['Price (€)'] <= budget)
    ]
    last_results = results
    return results.head(5)

def recommend_gifts():
    while True:
        recipient, gender, occasion, relationship, interest, budget = ask_gift_questions()
        if 'exit' in [recipient, gender, occasion, relationship, interest, budget]:
            print("🛑 Exiting gift recommendations.")
            break
        results = filter_gifts(recipient, gender, occasion, relationship, interest, budget)
        if not results.empty:
            print(f"{random.choice(fun_emojis)} Here are some amazing gift ideas for you! {random.choice(fun_emojis)}")
            show_gif()
            time.sleep(1)
            for idx, row in results.iterrows():
                print("")
                print(f"🎁 - {row['Product Name']}:")
                print(f"        - Description: {row['Product Description']}")
                print(f"        - Price: the price is ~{row['Price (€)']}€")
                print(f"        - Purchase link: {row['Purchase Link']}")
                time.sleep(0.5)
            ask_for_more_ideas()
            break
        else:
            print("😔 Sorry, no gifts matched your criteria.")
            retry = input("Would you like to try with different criteria? (yes/no): ").lower()
            if retry != 'yes':
                print("💡 Let me know if there's anything else I can help you with!")
                break

def ask_for_more_ideas():
    print("")
    more = input("Would you like to see more gift ideas? (yes/no): ").lower()
    if more == 'yes':
        show_more_gifts()

def show_more_gifts():
    global last_results
    more_results = last_results.iloc[5:]
    if not more_results.empty:
        print("")
        print(f"{random.choice(fun_emojis)} Here are more gift ideas for you! {random.choice(fun_emojis)}")
        show_gif()
        for idx, row in more_results.iterrows():
            print("")
            print(f"🎁 - {row['Product Name']}:")
            print(f"        - Description: {row['Product Description']}")
            print(f"        - Price: {row['Price (€)']}€")
            print(f"        - Purchase link: {row['Purchase Link']}")
            time.sleep(0.5)
        print("")
        print("✨ That's all the gift ideas matching your criteria! If nothing fits, you can refine your search.")
        print("Would you like to refine your criteria? (yes/no)")
        refine = input().lower()
        if refine == 'yes':
            recommend_gifts()
        else:
            print("💡 What else can I help you with?")
            general_chat(input("You: "))
    else:
        print("✨ No more gift ideas available! ✨")

def general_chat(user_message):
    faq_response = is_faq_related(user_message)
    if faq_response:
        print(faq_response)
        show_gif()
    else:
        response = chatbot(
            user_message,
            max_length=150,
            num_return_sequences=1,
            truncation=True,
            pad_token_id=50256,
            temperature=0.7,
            top_p=0.9,
            top_k=50,
            repetition_penalty=1.2
        )
        print(response[0]['generated_text'].strip())

def main():
    print(f"{random.choice(fun_emojis)} Hello! I'm your personal assistant. Ask me anything! {random.choice(fun_emojis)}")
    while True:
        print("")
        user_message = input("You: ")
        if user_message=='exit':
            print(f"👋 Goodbye! Have a fantastic day! {random.choice(fun_emojis)}")
            break
        elif is_gift_related(user_message):
            recommend_gifts()
        else:
            general_chat(user_message)

if __name__ == "__main__":
    main()